* 대회 데이터 셋 오류로 인한 데이터 전처리
  * https://dacon.io/competitions/official/235745/talkboard/403708?page=1&dtype=recent

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [2]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')
age = pd.read_csv('age_gender_info.csv')

train.shape, test.shape, sub.shape, age.shape

((2952, 15), (1022, 14), (150, 2), (16, 23))

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          1022 non-null   object 
 1   총세대수                          1022 non-null   int64  
 2   임대건물구분                        1022 non-null   object 
 3   지역                            1022 non-null   object 
 4   공급유형                          1022 non-null   object 
 5   전용면적                          1022 non-null   float64
 6   전용면적별세대수                      1022 non-null   int64  
 7   공가수                           1022 non-null   float64
 8   자격유형                          1020 non-null   object 
 9   임대보증금                         842 non-null    object 
 10  임대료                           842 non-null    object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  980 non-null    float64
 12  도보 10분거리 내 버스정류장 수            1022 non-null   float64
 13  단지내

In [5]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [7]:
test.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수'],
      dtype='object')

In [8]:
train.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', 
                 '공가수', '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', 
                 '단지내주차면수', '등록차량수']
test.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', 
                 '공가수', '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', 
                 '단지내주차면수']

In [9]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [10]:
test.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수'],
      dtype='object')

### 데이터 오류로 인한 데이터 제외
* test set에서 평가 제외되는 데이터(3개 단지): 'C2675'(이상 2번 오류), 'C2335', 'C1327'(이상 3번 오류)

#### 2번 오류
* [2번 오류] 동일한 단지에 코드가 2개로 부여된 단지 코드 (3쌍) : ['C2085', 'C1397'], ['C2431', 'C1649'], ['C1036', 'C2675(test)'] - C2675 제외 train set
* (참고 사항) 주차면수는 하나의 단지임을 전제로 산정된 것이고 총세대수는 두 개 단지의 합계입니다. 다만 등록차량대수는 ['C2085', 'C1397'] 단지의 경우 동일 수치

##### ['C1397', 'C2085']

In [11]:
train.loc[((train['단지코드']=='C1397')|(train['단지코드']=='C2085')), :]  # 장기전세는 임대료 없음
# ['C1397', 'C2085']단지의 '단지내주차면수'와 '등록차량수'는 각각 동일 수치(1590.0, 1595.0)로 들어감

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
2043,C1397,370,아파트,서울특별시,장기전세,23.32,51,9.0,A,87444000,-,0.0,3.0,1590.0,1595.0
2044,C1397,370,아파트,서울특별시,장기전세,46.79,4,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0
2045,C1397,370,아파트,서울특별시,장기전세,46.91,69,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0
2046,C1397,370,아파트,서울특별시,장기전세,51.96,24,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0
2047,C1397,370,아파트,서울특별시,장기전세,51.99,80,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0
2048,C1397,370,아파트,서울특별시,장기전세,59.93,142,9.0,A,216423000,-,0.0,3.0,1590.0,1595.0
2186,C2085,969,아파트,서울특별시,공공임대(10년),51.96,63,0.0,A,47001000,602270,0.0,3.0,1590.0,1595.0
2187,C2085,969,아파트,서울특별시,공공임대(10년),51.99,41,0.0,A,47001000,602270,0.0,3.0,1590.0,1595.0
2188,C2085,969,아파트,서울특별시,공공임대(10년),59.93,315,0.0,A,61210000,695170,0.0,3.0,1590.0,1595.0
2189,C2085,969,아파트,서울특별시,공공임대(분납),74.80,208,0.0,A,120611000,293490,0.0,3.0,1590.0,1595.0


* C1397과 C2085의 총세대수 합계(1339 = 370 + 969)로 두 단지코드의 총세대수를 변경

In [12]:
train.loc[train['단지코드']=='C2085', '총세대수'] = 1339
train.loc[train['단지코드']=='C1397', '총세대수'] = 1339

* C1397과 C2085의 단지코드를 N2085로 변경

In [13]:
print(train.loc[train['단지코드']=='C1397', :].shape)
print(train.loc[train['단지코드']=='C2085', :].shape)

(6, 15)
(8, 15)


In [14]:
train.loc[train['단지코드']=='C1397', '단지코드'] = 'N2085'
train.loc[train['단지코드']=='C2085', '단지코드'] = 'N2085'

In [15]:
train.loc[train['단지코드']=='N2085', :].shape

(14, 15)

In [16]:
train.loc[((train['단지코드']=='C1397')|(train['단지코드']=='C2085')), :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수


In [17]:
train.loc[(train['단지코드']=='N2085'), :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
2043,N2085,1339,아파트,서울특별시,장기전세,23.32,51,9.0,A,87444000,-,0.0,3.0,1590.0,1595.0
2044,N2085,1339,아파트,서울특별시,장기전세,46.79,4,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0
2045,N2085,1339,아파트,서울특별시,장기전세,46.91,69,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0
2046,N2085,1339,아파트,서울특별시,장기전세,51.96,24,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0
2047,N2085,1339,아파트,서울특별시,장기전세,51.99,80,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0
2048,N2085,1339,아파트,서울특별시,장기전세,59.93,142,9.0,A,216423000,-,0.0,3.0,1590.0,1595.0
2186,N2085,1339,아파트,서울특별시,공공임대(10년),51.96,63,0.0,A,47001000,602270,0.0,3.0,1590.0,1595.0
2187,N2085,1339,아파트,서울특별시,공공임대(10년),51.99,41,0.0,A,47001000,602270,0.0,3.0,1590.0,1595.0
2188,N2085,1339,아파트,서울특별시,공공임대(10년),59.93,315,0.0,A,61210000,695170,0.0,3.0,1590.0,1595.0
2189,N2085,1339,아파트,서울특별시,공공임대(분납),74.80,208,0.0,A,120611000,293490,0.0,3.0,1590.0,1595.0


##### ['C1649', 'C2431']

In [19]:
train.loc[((train['단지코드']=='C1649')|(train['단지코드']=='C2431')), :] 
# ['C1649', 'C2431'] 단지의 단지내주차면수 동일하게(1066.0) 들어감

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
2315,C1649,575,아파트,경상남도,공공임대(10년),74.97,80,15.0,A,46000000,456000,NaN,NaN,1066.0,855.0
2316,C1649,575,아파트,경상남도,공공임대(10년),84.95,124,15.0,A,57000000,462000,NaN,NaN,1066.0,855.0
2317,C1649,575,아파트,경상남도,공공임대(10년),84.96,289,15.0,A,57000000,462000,NaN,NaN,1066.0,855.0
2318,C1649,575,아파트,경상남도,공공임대(10년),84.98,82,15.0,A,57000000,462000,NaN,NaN,1066.0,855.0
2372,C2431,472,아파트,경상남도,국민임대,36.77,272,16.0,A,11217000,233330,0.0,2.0,1066.0,359.0
2373,C2431,472,아파트,경상남도,국민임대,46.78,200,16.0,A,24389000,303220,0.0,2.0,1066.0,359.0


* C1649, C2431의 총세대수 합계(1047 = 575 + 472)로 두 단지코드의 총세대수 변경

In [20]:
train.loc[train['단지코드']=='C1649', '총세대수'] = 1047
train.loc[train['단지코드']=='C2431', '총세대수'] = 1047

* C1649, C2431의 등록차량대수 합계(1214 = 359 + 855)로 두 단지코드의 등록차량대수 변경

In [21]:
train.loc[train['단지코드']=='C1649', '등록차량수'] = 1214
train.loc[train['단지코드']=='C2431', '등록차량수'] = 1214

* C1649와 C2431의 단지코드를 N2431로 변경

In [24]:
print(train.loc[train['단지코드']=='C1649', :].shape)
print(train.loc[train['단지코드']=='C2431', :].shape)

(4, 15)
(2, 15)


In [25]:
train.loc[train['단지코드']=='C1649', '단지코드'] = 'N2431'
train.loc[train['단지코드']=='C2431', '단지코드'] = 'N2431'

In [26]:
train.loc[train['단지코드']=='N2431', :].shape

(6, 15)

In [27]:
train.loc[train['단지코드']=='N2431', :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
2315,N2431,1047,아파트,경상남도,공공임대(10년),74.97,80,15.0,A,46000000,456000,NaN,NaN,1066.0,1214.0
2316,N2431,1047,아파트,경상남도,공공임대(10년),84.95,124,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0
2317,N2431,1047,아파트,경상남도,공공임대(10년),84.96,289,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0
2318,N2431,1047,아파트,경상남도,공공임대(10년),84.98,82,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0
2372,N2431,1047,아파트,경상남도,국민임대,36.77,272,16.0,A,11217000,233330,0.0,2.0,1066.0,1214.0
2373,N2431,1047,아파트,경상남도,국민임대,46.78,200,16.0,A,24389000,303220,0.0,2.0,1066.0,1214.0


#### ['C1036', 'C2675(test set)']

In [29]:
train.loc[train['단지코드']=='C1036']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
1502,C1036,742,아파트,경기도,국민임대,36.65,135,13.0,A,18476000,154790,0.0,3.0,1016.0,1279.0
1503,C1036,742,아파트,경기도,국민임대,46.90,45,13.0,A,34082000,232200,0.0,3.0,1016.0,1279.0
1504,C1036,742,아파트,경기도,국민임대,46.90,171,13.0,A,34082000,232200,0.0,3.0,1016.0,1279.0
1505,C1036,742,아파트,경기도,국민임대,46.90,81,13.0,A,34082000,232200,0.0,3.0,1016.0,1279.0
1506,C1036,742,아파트,경기도,국민임대,46.90,90,13.0,A,34082000,232200,0.0,3.0,1016.0,1279.0
1507,C1036,742,아파트,경기도,국민임대,51.93,184,13.0,A,43819000,275900,0.0,3.0,1016.0,1279.0
1508,C1036,742,아파트,경기도,국민임대,51.93,36,13.0,A,43819000,275900,0.0,3.0,1016.0,1279.0


In [30]:
test.loc[test['단지코드']=='C2675']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
579,C2675,512,아파트,경기도,국민임대,36.65,130,9.0,A,18476000,154790,0.0,3.0,1016.0
580,C2675,512,아파트,경기도,국민임대,46.90,44,9.0,A,34082000,232200,0.0,3.0,1016.0
581,C2675,512,아파트,경기도,국민임대,46.90,80,9.0,A,34082000,232200,0.0,3.0,1016.0
582,C2675,512,아파트,경기도,국민임대,46.90,50,9.0,A,34082000,232200,0.0,3.0,1016.0
583,C2675,512,아파트,경기도,국민임대,46.90,88,9.0,A,34082000,232200,0.0,3.0,1016.0
584,C2675,512,아파트,경기도,국민임대,47.40,30,9.0,A,34082000,232200,0.0,3.0,1016.0
585,C2675,512,아파트,경기도,국민임대,51.93,75,9.0,A,43819000,275900,0.0,3.0,1016.0
586,C2675,512,아파트,경기도,국민임대,51.93,15,9.0,A,43819000,275900,0.0,3.0,1016.0


In [31]:
train.loc[train['단지코드']=='C2675']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수


C2675 단지코드는 test set에서 평가 제외되므로 삭제

In [32]:
test = test.loc[~(test['단지코드']=='C2675'), :]
test

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0
1018,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0
1019,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0
1020,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0


In [33]:
train.loc[train['단지코드']=='C1036'].shape

(7, 15)

* C1036의 총세대수를 1254로 변경

In [34]:
train.loc[train['단지코드']=='C1036', '총세대수'] = 1254

* C1036의 단지코드를 N1036으로 변경

In [35]:
train.loc[train['단지코드']=='C1036', '단지코드'] = 'N1036'

In [36]:
train.loc[train['단지코드']=='N1036'].shape

(7, 15)

In [37]:
train.loc[train['단지코드']=='N1036']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
1502,N1036,1254,아파트,경기도,국민임대,36.65,135,13.0,A,18476000,154790,0.0,3.0,1016.0,1279.0
1503,N1036,1254,아파트,경기도,국민임대,46.90,45,13.0,A,34082000,232200,0.0,3.0,1016.0,1279.0
1504,N1036,1254,아파트,경기도,국민임대,46.90,171,13.0,A,34082000,232200,0.0,3.0,1016.0,1279.0
1505,N1036,1254,아파트,경기도,국민임대,46.90,81,13.0,A,34082000,232200,0.0,3.0,1016.0,1279.0
1506,N1036,1254,아파트,경기도,국민임대,46.90,90,13.0,A,34082000,232200,0.0,3.0,1016.0,1279.0
1507,N1036,1254,아파트,경기도,국민임대,51.93,184,13.0,A,43819000,275900,0.0,3.0,1016.0,1279.0
1508,N1036,1254,아파트,경기도,국민임대,51.93,36,13.0,A,43819000,275900,0.0,3.0,1016.0,1279.0


* [2번 오류] test set의 C2675 단지코드(8 x 14) 삭제 완료, test set shape -> (1014 x 14)

#### 3번 오류
```
단지코드 등 기입 실수로 데이터 정제 과정에서 매칭 오류 발생  
 - (오류 내용) 단지코드 등 기입 실수로 총세대수가 주차면수에 비해 과하게 많거나 적은 경우가 발생하였고, 점검 결과 일부 데이터의 단지코드, 총세대수, 주차면수 등에서 오류 검출
 - (발생 원인) 원천데이터 수집 과정에서 단지 코드 등이 잘못 기입되었고 이를 인지하지 못한 채 데이터 정제를 하여 오류 발생
 - (관련 데이터) 아래와 같이 총 9개 단지에서 같은 문제 확인 
※ 실수가 발생한 단지 코드 (9개 단지) : ['C2335', 'C1327', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
 - C2335, C1327 단지는 테스트셋, 나머지는 트레인셋
```

##### ['C2335', 'C1327'] (test data set) 단지코드 삭제

In [38]:
test = test.loc[~((test['단지코드']=='C2335')|(test['단지코드']=='C1327')), :]
test

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0
1018,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0
1019,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0
1020,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0


test set 오류 삭제 확인

In [39]:
test.loc[((test['단지코드']=='C2675')|(test['단지코드']=='C2335')|(test['단지코드']=='C1327')), :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수


##### ['C1095', 'C1218', 'C1502', 'C1894', 'C1988', 'C2051', 'C2483'] (train set) 오류 처리

In [40]:
train.loc[((train['단지코드']=='C1095')|(train['단지코드']=='C1218')|(train['단지코드']=='C1502')|
         (train['단지코드']=='C1894')|(train['단지코드']=='C1988')|(train['단지코드']=='C2051')|
         (train['단지코드']=='C2483')), :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
5,C2483,900,아파트,경상북도,국민임대,51.95,89,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
6,C2483,900,아파트,경상북도,국민임대,51.95,135,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
7,C2483,900,아파트,경상북도,국민임대,59.88,86,38.0,A,30357000,214270,0.0,3.0,1425.0,1015.0
1228,C1988,475,아파트,전라남도,국민임대,36.63,200,12.0,A,12026000,87940,0.0,2.0,722.0,402.0
1229,C1988,475,아파트,전라남도,국민임대,36.63,43,12.0,A,12026000,87940,0.0,2.0,722.0,402.0


* train 데이터 셋에 오류 발생 코드를 ERR04로 변경 후, 데이터 셋을 두개로 분리

In [41]:
train.loc[train['단지코드']=='C1095', '단지코드'] = 'ERR04_1095'
train.loc[train['단지코드']=='C1218', '단지코드'] = 'ERR04_1218'
train.loc[train['단지코드']=='C1502', '단지코드'] = 'ERR04_1502'
train.loc[train['단지코드']=='C1894', '단지코드'] = 'ERR04_1894'
train.loc[train['단지코드']=='C1988', '단지코드'] = 'ERR04_1988'
train.loc[train['단지코드']=='C2051', '단지코드'] = 'ERR04_2051'
train.loc[train['단지코드']=='C2483', '단지코드'] = 'ERR04_2483'

In [42]:
train.loc[train['단지코드'].str.contains('ERR'), :].shape

(56, 15)

In [43]:
train.loc[train['단지코드'].str.contains('ERR'), :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
0,ERR04_2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,ERR04_2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,ERR04_2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,ERR04_2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,ERR04_2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
5,ERR04_2483,900,아파트,경상북도,국민임대,51.95,89,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
6,ERR04_2483,900,아파트,경상북도,국민임대,51.95,135,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
7,ERR04_2483,900,아파트,경상북도,국민임대,59.88,86,38.0,A,30357000,214270,0.0,3.0,1425.0,1015.0
1228,ERR04_1988,475,아파트,전라남도,국민임대,36.63,200,12.0,A,12026000,87940,0.0,2.0,722.0,402.0
1229,ERR04_1988,475,아파트,전라남도,국민임대,36.63,43,12.0,A,12026000,87940,0.0,2.0,722.0,402.0


In [44]:
train_df_errno = train.loc[~train['단지코드'].str.contains('ERR'), :]
train_df_errno

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
8,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0,205.0
9,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0,205.0
10,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,107130,0.0,3.0,624.0,205.0
11,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,149760,0.0,3.0,624.0,205.0
12,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,149760,0.0,3.0,624.0,205.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0


### 데이터 오류 처리 후 csv 파일 만들기

In [45]:
train_df = train.copy()
test_df = test.copy()

In [46]:
train_df.to_csv('train_df.csv', index=False)
train_df_errno.to_csv('train_df_errno.csv', index=False)
test_df.to_csv('test_df.csv', index=False)

* train_df.csv : 오류 2번 해결, 오류 3번은 ERR 코드를 붙여 남겨놓음
* train_df_errno.csv : 오류 2번, 3번 해결
* test_df.csv : test set 오류 코드 제외 csv 파일